In [4]:
import os
import pandas as pd
import numpy as np
from utils.utils import *
import time

In [5]:
sior = {v:k for k, v in rois.items()}
sior

{1: 'V1',
 2: 'V2',
 3: 'V3',
 4: 'hV4',
 5: 'VO-1',
 6: 'VO-2',
 7: 'PHC-1',
 8: 'PHC-2',
 9: 'LO-1',
 10: 'LO-2',
 11: 'TO-1',
 12: 'TO-2'}

In [6]:
subjects_sessions

{1: ('subj01', 40),
 2: ('subj02', 40),
 3: ('subj03', 32),
 4: ('subj04', 30),
 5: ('subj05', 40),
 6: ('subj06', 32),
 7: ('subj07', 40),
 8: ('subj08', 30)}

In [9]:
for i, (sub, n_sessions) in subjects_sessions.items():
    start_sub = time.time()
    print(f'Enter {sub} at {time.strftime("%H:%M:%S", time.gmtime(start_sub))}')
    maskdata_file = os.path.join(data_dir, 'mask', sub, f'short.reduced.{sub}.testrois.npy') # normal mask break everything
    maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)
    n_voxels = maskdata.shape[0]

    belongs = []
    print(maskdata)
    for i in maskdata:
        belongs.append(sior[i])
      #  print(sior[i])

    columns = ["x0", "y0", "sigma", "slope", "intercept",  "var_explained", "mds_ecc", "mds_ang"]

    def build_model(model):
        model_out = pd.DataFrame(np.zeros((n_voxels, len(columns))), columns=columns, dtype="Float32")
        model_out.var_explained = -np.inf
        model_out["roi"] = belongs
        model_out["fit_with"] = -1
        for roi_name, roi_value in rois.items():
            fits_ss_file = os.path.join(
                proj_dir, 'fits_not_rotated', sub, f'fits_{sub}_{n_sessions}_trains_version2_{roi_name}.npy'
            )
            fits_ss = pd.DataFrame(np.load(fits_ss_file, allow_pickle=True), columns=columns, dtype = "Float32")
            fits_ss["roi"] = belongs  ### IS THAT THE ISSUE????????????? 
            fits_ss["fit_with"] = roi_name


            match model:
                case 'wself':
                    fit_mask = model_out.var_explained < fits_ss.var_explained
                case 'oself':
                    only_self = fits_ss.roi == fits_ss.fit_with
                    fit_mask = model_out.var_explained < fits_ss.var_explained
                    fit_mask = np.logical_and(only_self, fit_mask)
            if any(fit_mask):
                update_fits = fits_ss[fit_mask]
                model_out.update(update_fits)
            print(model_out.roi)
        return model_out

    models = ['oself']
    models_files = {}
    all_exist = True
    for m in models:
        model_file = os.path.join(proj_dir, 'models', sub, f'best_fits_{m}_{sub}_{n_sessions}_NR.npy')
        if not os.path.exists(model_file):
            model_out = build_model(m)
            for roi in rois.keys():
                print(f'{roi}: {model_out.loc[model_out.roi == roi]}')
            np.save(model_file, model_out)
            print(f'\t\t\t{m} saved')
        else:
            model_out = build_model(m)
            for roi in rois.keys():
                print(f'{roi}: {model_out.loc[model_out.roi == roi]}')
            np.save(model_file, model_out)
            print(f'\t\t\t{m} saved')
            
            
            print(f'\t\t\t{m} already saved')
            
            


Enter subj01 at 16:13:16
[1 1 1 ... 2 2 2]
0        V1
1        V1
2        V1
3        V1
4        V1
         ..
33792    V1
33793    V2
33794    V2
33795    V2
33796    V2
Name: roi, Length: 33797, dtype: object
0        V1
1        V1
2        V1
3        V1
4        V1
         ..
33792    V1
33793    V2
33794    V2
33795    V2
33796    V2
Name: roi, Length: 33797, dtype: object
0        V1
1        V1
2        V1
3        V1
4        V1
         ..
33792    V1
33793    V2
33794    V2
33795    V2
33796    V2
Name: roi, Length: 33797, dtype: object
0        V1
1        V1
2        V1
3        V1
4        V1
         ..
33792    V1
33793    V2
33794    V2
33795    V2
33796    V2
Name: roi, Length: 33797, dtype: object
0        V1
1        V1
2        V1
3        V1
4        V1
         ..
33792    V1
33793    V2
33794    V2
33795    V2
33796    V2
Name: roi, Length: 33797, dtype: object
0        V1
1        V1
2        V1
3        V1
4        V1
         ..
33792    V1
33793    V2
3

FileNotFoundError: [Errno 2] No such file or directory: '/home/stan/thesis-repo/data/mask/subj02/short.reduced.subj02.testrois.npy'